In [103]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor


In [9]:
dataset_url="https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv"
data = pd.read_csv(dataset_url)
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [57]:
# t=data.Month

In [64]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
series = array(data)
print(series.shape)
# transform to a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)

(144, 2)
(141, 3, 2) (141, 2)


In [76]:
# define univariate time series
passengers = data['Passengers'].values
# transform to a supervised learning problem
X, y = split_sequence(passengers, 2)
print(X.shape, y.shape)
dfX = pd.DataFrame(X, columns=["Xt-2", "Xt-1"])
dfy = pd.DataFrame(y, columns=["Xt"])

df = pd.concat((dfX, dfy), axis = 1)
df

(142, 2) (142,)


,Xt-2,Xt-1,Xt
0,112,118,132
1,118,132,129
2,132,129,121
3,129,121,135
4,121,135,148
...,...,...,...
137,535,622,606
138,622,606,508
139,606,508,461
140,508,461,390


In [96]:
df.iloc[::-1]

,Xt-2,Xt-1,Xt
141,461,390,432
140,508,461,390
139,606,508,461
138,622,606,508
137,535,622,606
...,...,...,...
4,121,135,148
3,129,121,135
2,132,129,121
1,118,132,129


##KNN

In [97]:
#mengambil nama kolom
nama_colm = df.columns.copy() 
#menghitung hasil normalisasi + menampilkan
scaler = MinMaxScaler()
scaler.fit(df)
X_normal = scaler.transform(df)
hasil = pd.DataFrame(X_normal,columns=nama_colm)
hasil.iloc[::-1]

,Xt-2,Xt-1,Xt
141,0.689189,0.552124,0.633205
140,0.779923,0.689189,0.552124
139,0.969112,0.779923,0.689189
138,1.000000,0.969112,0.779923
137,0.832046,1.000000,0.969112
...,...,...,...
4,0.032819,0.059846,0.084942
3,0.048263,0.032819,0.059846
2,0.054054,0.048263,0.032819
1,0.027027,0.054054,0.048263


In [98]:
hasil.describe()

,Xt-2,Xt-1,Xt
count,142.000000,142.000000,142.000000
mean,0.336791,0.340570,0.344839
std,0.231210,0.230304,0.230069
min,0.000000,0.000000,0.000000
25%,0.146718,0.147201,0.149614
50%,0.308880,0.311776,0.316602
75%,0.490830,0.493726,0.497104
max,1.000000,1.000000,1.000000


In [99]:
# ambil data
X = hasil.drop(columns="Xt")
y = hasil.Xt

In [102]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)
print(X_train.shape)
print(X_test.shape)

(113, 2)
(29, 2)


In [107]:
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train, y_train)
print("Hasil Akurasi Data Training Menggunakan KNN sebesar : ", (neigh.score(X_train, y_train)))
print("Hasil Akurasi Data Testing Menggunakan KNN sebesar : ", (neigh.score(X_test, y_test)))

Hasil Akurasi Data Training Menggunakan KNN sebesar :  0.9311427812397141
Hasil Akurasi Data Testing Menggunakan KNN sebesar :  0.9083353663906897
